<a href="https://colab.research.google.com/github/PG-1012/ML_Movie_Recommender/blob/main/PG-1012%20/%20ML_Movie_Recommender/Movie_Recommender_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the Surprise library
!pip install surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163759 sha256=690a88886ae38ca6e5725b4e477b1cc6762d52f3bf32d3c8c87e6818f5c3fab7
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
# Import the necessary libraries
import surprise
from surprise import SVD
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, train_test_split
from surprise.prediction_algorithms import SVD, KNNBasic, KNNBaseline
from surprise import accuracy, Dataset, SVD, NMF
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import random as rand
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
# Mount google drive to google colab so that data files can be accessed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load user ratings data
udata = pd.read_csv('/content/drive/Shareddrives/ITM760/u.data', sep='\t', names=["user id", "item id", "rating", "timestamp"])

In [ ]:
# Load genre data
ugenre = pd.read_csv('/content/drive/Shareddrives/ITM760/u.genre', sep="|", names=["Genre", "Index"])

In [ ]:
# Load movie csv data, declaring seperators and column names
uitem = pd.read_csv('/content/drive/Shareddrives/ITM760/u.item', sep="|", names=["movie id", "movie title", "release date", "video release date"
,"IMDb URL", "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
"Musical", "Mystery","Romance", "Sci-Fi","Thriller", "War","Western"], encoding="ISO-8859-1")

In [ ]:
# Preivew user ratings data
udata

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [ ]:
# Preview movie data
uitem.head(10)

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,9,Dead Man Walking (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Dead%20Man%20...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,Richard III (1995),22-Jan-1996,NaN,http://us.imdb.com/M/title-exact?Richard%20III...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Drop unnecessary columns that the algorithim will not be using
uitem_simp = uitem.drop(columns=['release date', 'video release date', 'IMDb URL'])

In [ ]:
# Preview new table that has removed columns
uitem_simp

,movie id,movie title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Preview genre data
ugenre.head(10)

,Genre,Index
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4
5,Comedy,5
6,Crime,6
7,Documentary,7
8,Drama,8
9,Fantasy,9


In [ ]:
# creates a reader for understanding the contents of the udata file
reader = Reader(line_format=u'user item rating timestamp', sep='\t', rating_scale=(1, 6), skip_lines=1)

In [ ]:
#uses the reader to read and load data from u.data
data = Dataset.load_from_file("/content/drive/Shareddrives/ITM760/u.data", reader=reader)

In [ ]:
# Determine the average rating using info in rating column of table udata
average = udata['rating'].mean()

In [ ]:
# Split dataset into training and testing
trainset, testset = train_test_split(data, test_size=0.25) #Please confirm if this is done correctly. Should it be x_train, x_test, y_train, y_test?

In [ ]:
#SVD:
algo_SVD=SVD()
algo_SVD.fit(trainset) #training the algorithm
predictions_SVD=algo_SVD.test(testset) #getting the predictions
RMSE_SVD = accuracy.rmse(predictions_SVD) #finding the RMSE
RMSE_SVD
accuracy.mae(predictions_SVD) #finding the MAE
accuracy.fcp(predictions_SVD) #finding the FCP


RMSE: 0.9401
MAE:  0.7424
FCP:  0.7039


0.7038692023777405

In [ ]:
threshold = average # Threshold based on average rating by users. ######## PUT IN EXPERIMENT SECTION
binary_predictions_SVD = [] #empty list to hold predictions

actual_ratings_SVD = []

for pred in predictions_SVD: #for loop that determines if value is 1 or 0 based on threshold
  if pred.est > threshold:
    binary_predictions_SVD.append(1)
  else:
    binary_predictions_SVD.append(0)

for (_, _, rating) in testset: #changing the ratings into binary
  if rating > threshold:
    actual_ratings_SVD.append(1)
  else:
    actual_ratings_SVD.append(0)

# Calculating the precision, recall, and F1 score for SVD
precision, recall, f1_SVD, _ = precision_recall_fscore_support(binary_predictions_SVD, actual_ratings_SVD, average='weighted') ## why are we using weighted?

In [ ]:
f1_SVD

0.7062423641945076

In [ ]:
#KNN
algo_KNN=KNNBasic()
algo_KNN.fit(trainset) #training the algorithm
predictions_KNN=algo_KNN.test(testset) #getting the predictions
RMSE_KNN = accuracy.rmse(predictions_KNN) #finding the RMSE
RMSE_KNN
accuracy.mae(predictions_KNN) #finding the MAE
accuracy.fcp(predictions_KNN) #finding the FCP

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9871
MAE:  0.7795
FCP:  0.7159


0.7158862789322178

In [ ]:
threshold = average  # Adjust this threshold based on your problem. ######## PUT IN EXPERIMENT SECTION

binary_predictions_KNN = []

actual_ratings_KNN = []

for pred in predictions_KNN: #for loop that determines if value is 1 or 0 based on threshold
  if pred.est > threshold:
    binary_predictions_KNN.append(1)
  else:
    binary_predictions_KNN.append(0)

for (_, _, rating) in testset: #changing the ratings into binary
  if rating > threshold:
    actual_ratings_KNN.append(1)
  else:
    actual_ratings_KNN.append(0)

# Calculating the precision, recall, and F1 score for SVD
precision, recall, f1_KNN, _ = precision_recall_fscore_support(binary_predictions_KNN, actual_ratings_KNN, average='weighted')

In [ ]:
f1_KNN

0.6923131602796273

In [ ]:
#NMF
algo_NMF=NMF()
algo_NMF.fit(trainset) #training the algorithm
predictions_NMF=algo_NMF.test(testset) #getting the predictions
RMSE_NMF = accuracy.rmse(predictions_NMF) #finding the RMSE
RMSE_NMF
accuracy.mae(predictions_NMF)#finding the MAE
accuracy.fcp(predictions_NMF)#finding the FCP

RMSE: 0.9699
MAE:  0.7633
FCP:  0.6971


0.697057060570308

In [ ]:
threshold = average  # Adjust this threshold based on your problem. ######## PUT IN EXPERIMENT SECTION

binary_predictions_NMF = []

actual_ratings_NMF = []


for pred in predictions_NMF: #for loop that determines if value is 1 or 0 based on threshold
  if pred.est > threshold:
    binary_predictions_NMF.append(1)
  else:
    binary_predictions_NMF.append(0)

for (_, _, rating) in testset:  #changing the ratings into binary
  if rating > threshold:
    actual_ratings_NMF.append(1)
  else:
    actual_ratings_NMF.append(0)

# Calculate precision, recall, and F1 score
precision, recall, f1_NMF, _ = precision_recall_fscore_support(binary_predictions_NMF, actual_ratings_NMF, average='weighted')

In [ ]:
f1_NMF

0.6943442118852424

In [ ]:
best_f1 = max(f1_KNN, f1_NMF, f1_SVD) #max f1
best_RMSE = min(RMSE_NMF, RMSE_KNN, RMSE_SVD) #min RMSE
print(best_f1)
print(best_RMSE) # THIS GETS THE LOWEST RMSE AND HIGHEST F1 SCORE, POINTS TO SVD BEING THE BEST ALGORITHM

0.7062423641945076
0.9400728926601573


In [ ]:
uitem_simp = uitem.drop(columns=['video release date', 'IMDb URL']) #drop the video release date and the IMDb URL

In [ ]:
movies_df = pd.DataFrame(uitem_simp) #create a data frame
movies_df

,movie id,movie title,release date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
user_list = [196, 200, 102, 294, 299]

for user_id_to_recommend in user_list: #for loop ti get through all users
  length = 10
  SVD_predictions = algo_SVD.test(testset) #get predictions from the testset for SVD
  target_id = udata.loc[udata['user id'] == user_id_to_recommend]
  index_difference = set(udata['item id']) - set(target_id['item id']) #find the difference to find the unrated movies
  target_id_list = list(index_difference)

  predicted_ratings = [] #initialize empty list
  for movie_id in target_id_list: #for loop to go through target_id_list
    prediction = algo_SVD.predict(user_id_to_recommend, movie_id) #get predictions
    predicted_ratings.append(prediction.est) #append predictions

  pairs = zip(target_id_list, predicted_ratings) #create tuples with the target id and prediction values
  sorted_ratings = sorted(list(pairs), key=lambda x: x[1], reverse=True) #sort the ratings

  top_movies = [] #initialize empty list
  for movie_id, _ in sorted_ratings[:length]: #for loop going through the sorted ratings
    top_movies.append(movie_id) #appending the movie id's

  print(f"Top 10 recommended movies for user {user_id_to_recommend}:" + str(top_movies))

  for movie in top_movies:
      print(movies_df.at [ movie, "movie title"])

Top 10 recommended movies for user 196:[1, 2, 3, 4, 5, 6, 7, 9, 10, 11]
GoldenEye (1995)
Four Rooms (1995)
Get Shorty (1995)
Copycat (1995)
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
Twelve Monkeys (1995)
Babe (1995)
Richard III (1995)
Seven (Se7en) (1995)
Usual Suspects, The (1995)
Top 10 recommended movies for user 200:[3, 4, 5, 6, 10, 12, 13, 14, 16, 17]
Get Shorty (1995)
Copycat (1995)
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
Twelve Monkeys (1995)
Seven (Se7en) (1995)
Mighty Aphrodite (1995)
Postino, Il (1994)
Mr. Holland's Opus (1995)
From Dusk Till Dawn (1996)
White Balloon, The (1995)
Top 10 recommended movies for user 102:[3, 6, 8, 9, 10, 12, 14, 15, 16, 17]
Get Shorty (1995)
Twelve Monkeys (1995)
Dead Man Walking (1995)
Richard III (1995)
Seven (Se7en) (1995)
Mighty Aphrodite (1995)
Mr. Holland's Opus (1995)
French Twist (Gazon maudit) (1995)
From Dusk Till Dawn (1996)
White Balloon, The (1995)
Top 10 recommended movies for user 294:[2, 3, 4, 5, 6, 8, 9, 

In [ ]:
features = movies_df.drop(["movie id", "movie title", "release date"], axis=1) #droping non numerical values
features

,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features) #standardizes the fetures to a one format.

In [ ]:
cosine_sim = cosine_similarity(features_standardized) #calculating the cosine similarities based on the standardized features.

In [ ]:
def get_top_similar_movies(movie_id, cosine_sim, movies_df, top=10):

    movie_id_check = movies_df.index[movies_df["movie id"] == movie_id]
    index_match = movie_id_check.tolist()[0]

    enum = enumerate(cosine_sim[index_match]) #enumerate cosine_sim at the index_match
    sim_scores = list(enum)
    sim_score_srted = sorted(sim_scores, key=lambda x: x[1], reverse=True) #sorting
    actual_len = top + 1
    top_similar_movies = sim_score_srted[1:actual_len]  # beginning from 1 excludes the same movie that is being checked.
    top_movie_indexs = []
    for index, _ in top_similar_movies: #for loop to loop through index in top_similar_movies
      top_movie_indexs.append(index) #append index

    return movies_df["movie title"][top_movie_indexs] #return the top movies indexes

In [ ]:
  list_of_movie_id = [174, 678, 1217, 540, 474]
for x in list_of_movie_id: #for loop to go through all user id's
    movie_id_to_check = x
    top_similar_movies = get_top_similar_movies(movie_id_to_check, cosine_sim, movies_df) #calling the function
    print(f"Top 10 similar movies to the movie {movie_id_to_check}:\n{top_similar_movies}")

Top 10 similar movies to the movie 174:
173               Raiders of the Lost Ark (1981)
209    Indiana Jones and the Last Crusade (1989)
265                    Kull the Conqueror (1997)
430                            Highlander (1986)
490         Adventures of Robin Hood, The (1938)
540                         Mortal Kombat (1995)
553                            Waterworld (1995)
678                   Conan the Barbarian (1981)
679                    Kull the Conqueror (1997)
828                                  Fled (1996)
Name: movie title, dtype: object
Top 10 similar movies to the movie 678:
42                    Disclosure (1994)
43             Dolores Claiborne (1994)
76                     Firm, The (1993)
97     Silence of the Lambs, The (1991)
105                   Diabolique (1996)
222                  Sling Blade (1996)
314                    Apt Pupil (1998)
324                        Crash (1996)
369                  Mary Reilly (1996)
580                   Kalifornia (199